In [ ]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/RIS-LAD",
    overwrite=True,
    persistent=True
)

In [ ]:
dataset

In [ ]:
dataset.first()

In [ ]:
dataset.count("ground_truth.detections.label")

In [ ]:
dataset.count_values("ground_truth.detections.label")

In [ ]:
dataset.count("ground_truth.detections.referring_expression")

In [ ]:
len(dataset.distinct("ground_truth.detections.referring_expression"))

In [ ]:
import fiftyone.zoo as foz

# Register this custom model source
foz.register_zoo_model_source("https://github.com/harpreetsahota204/siglip2", overwrite=True)

In [ ]:
import fiftyone.zoo as foz

siglip_model = foz.load_zoo_model(
    "google/siglip2-giant-opt-patch16-256"
)

In [ ]:
dataset.compute_embeddings(
    model=siglip_model,
    embeddings_field="siglip2_embeddings",
)

In [ ]:
import fiftyone.brain as fob

results = fob.compute_visualization(
    dataset,
    embeddings="siglip2_embeddings",
    method="umap",
    brain_key="siglip2_viz",
    num_dims=2,
)


In [ ]:
# Build a similarity index
text_img_index = fob.compute_similarity(
    dataset,
    model="google/siglip2-giant-opt-patch16-256",
    embeddings="siglip2_embeddings",
    brain_key="siglip2_similarity",
)

In [ ]:
session = fo.launch_app(dataset, auto=False)
session.url

In [ ]:
siglip_model.text_prompt = "Low altitude drone footage taken at "
siglip_model.classes = ["day", "night", "dusk"]

dataset.apply_model(
    siglip_model,
    label_field="time_of_day"
)

In [ ]:
siglip_model.text_prompt = "The scene in this low altitude drone footage is in a "
siglip_model.classes = ["urban area", "near water", "highway", "pedestrian area"]

dataset.apply_model(
    siglip_model,
    label_field="location"
)

In [ ]:
import fiftyone.zoo as foz

# Register the remote model source
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/sam3_images"
)

# Load the model
sam3_model = foz.load_zoo_model("facebook/sam3")

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob


# ============================================================
# 1. Compute Embeddings for Similarity Search
# ============================================================
sam3_model.pooling_strategy = "max"  # or "mean", "cls"

dataset.compute_embeddings(
    sam3_model,
    embeddings_field="sam_embeddings",
    batch_size=32
)

# Visualize with UMAP
fob.compute_visualization(
    dataset,
    method="umap",
    brain_key="sam_viz",
    embeddings="sam_embeddings",
    num_dims=2
)



In [ ]:

# ============================================================
# 4. Concept Segmentation (Find Multiple Object Types)
# ============================================================
sam3_model.operation = "concept_segmentation"
sam3_model.threshold = 0.5
sam3_model.mask_threshold = 0.5

dataset.apply_model(
    sam3_model,
    prompt_field="prompts",
    label_field="sam3_no_finetune",
    batch_size=8,
    num_workers=4
)

# Launch app
session = fo.launch_app(dataset)